In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Ruta de la carpeta que contiene los archivos CSV de entrenamiento
folder_path = r'C:\Users\Miguel Angel\Desktop\spaceapp\space_apps_2024_seismic_detection\data\lunar\training\data'

# Inicializamos una lista para almacenar los DataFrames
dataframes = []

# Recorremos cada subcarpeta y archivo CSV dentro de la carpeta principal
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(root, file_name)
            df = pd.read_csv(file_path)
            
            # Verificar si el archivo CSV tiene datos
            if not df.empty:
                dataframes.append(df)
            else:
                print(f"Advertencia: El archivo '{file_name}' está vacío.")

# Verificar si se encontraron archivos CSV
if not dataframes:
    print("No se pudo cargar ningún archivo CSV con datos.")
else:
    # Combinar todos los DataFrames en uno solo
    df = pd.concat(dataframes, ignore_index=True)

    # Ordenar los datos por tiempo absoluto
    df = df.sort_values('time_abs(%Y-%m-%dT%H:%M:%S.%f)')

    # Convertir la columna de tiempo absoluto a formato de fecha y hora
    df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'] = pd.to_datetime(df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])

    # ---------------------------------------------
    # Preparación de las características (features)
    # ---------------------------------------------
    # Parámetros para el cálculo de STA/LTA
    sampling_rate = 6  # Número de muestras por segundo (ajústalo según tus datos)
    sta_window = 1 * sampling_rate  # Ventana corta de 1 segundo
    lta_window = 10 * sampling_rate  # Ventana larga de 10 segundos

    # Calcular la media móvil (STA y LTA)
    sta = df['velocity(m/s)'].rolling(window=sta_window).mean()
    lta = df['velocity(m/s)'].rolling(window=lta_window).mean()

    # Evitar división por cero en la LTA reemplazando ceros por NaN
    lta = lta.replace(0, np.nan)

    # Relación STA/LTA
    df['sta_lta_ratio'] = sta / lta

    # Extraer características adicionales
    df['velocity_change'] = df['velocity(m/s)'].diff()  # Cambio en la velocidad
    df['time_delta'] = df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].diff().dt.total_seconds()  # Tiempo entre muestras

    # ---------------------------------------------
    # Definir las etiquetas (target)
    # ---------------------------------------------
    # Asumiremos que un evento sísmico ocurre cuando STA/LTA supera un umbral
    threshold = 3
    df['event'] = (df['sta_lta_ratio'] > threshold).astype(int)

    # Eliminar valores NaN resultantes de la media móvil
    df = df.dropna()

    # Seleccionar características para el modelo
    features = ['velocity(m/s)', 'sta_lta_ratio', 'velocity_change', 'time_delta']
    X = df[features]
    y = df['event']  # Etiqueta: si ocurrió o no un evento sísmico

    # Dividir los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # ---------------------------------------------
    # Entrenar el modelo de Random Forest
    # ---------------------------------------------
    model = RandomForestClassifier(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)

    # ---------------------------------------------
    # Evaluación del modelo
    # ---------------------------------------------
    # Predecir en los datos de prueba
    y_pred = model.predict(X_test)

    # Evaluar la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nPrecisión del modelo: {accuracy * 100:.2f}%")

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    print("\nMatriz de confusión:")
    print(cm)

    # Reporte de clasificación
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))


KeyboardInterrupt: 